In [1]:
import sagemaker
from time import gmtime, strftime
from sagemaker.spark.processing import PySparkProcessor

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
print(role)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123


### for test only

In [ ]:
!aws s3 cp s3://sagemaker-us-west-2-687912291502/mv-poc/raw/part-00041-464b88c1-fdba-4e52-bc1e-bc73314c1e77-c000.snappy.orc ./

In [ ]:
import json

def is_json(line):
    try:
        json.loads(line)
        return True
    except json.JSONDecodeError:
        return False

def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, 1):
            line = line.strip()
            line_data = json.loads(line)
            inner_json = json.loads(line_data["json"])
            for k, v in inner_json.items():
                #if k == "_12004":
                    print("k,v",k,v[1])

            
            

# 使用示例
file_path = './part-00000-1afde205-86bb-4fcc-a7d3-16d0137e2baf-c000.json'  # 替换为你的文件路径
process_file(file_path)


In [ ]:
!aws s3 cp s3://mv-mtg-di-for-poc/2024/06/16/08/part-01309-b0cfc571-2e0c-4445-8ecc-bbe6222436c7-c000.snappy.orc ./

In [ ]:
import pyarrow.orc as orc
import boto3
from io import BytesIO

# S3 bucket 和文件路径
bucket = "sagemaker-us-west-2-687912291502"
file_key = "mv-poc/raw3/2024/06/14/00/part-00013-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc"
# 创建 S3 客户端
s3 = boto3.client('s3')

# 从 S3 下载文件
response = s3.get_object(Bucket=bucket, Key=file_key)
file_content = response['Body'].read()

# 使用 BytesIO 创建一个类文件对象
file_obj = BytesIO(file_content)


# 打开 ORC 文件
table = orc.read_table(file_obj)

# 获取 schema
schema = table.schema

# 打印每个列的名称和类型
#for field in schema:
#    print(f"Column Name: {field.name}, Type: {field.type}")

# 读取第一行数据（如果文件不为空）
if table.num_rows > 0:
    first_row = table.slice(0, 1).to_pydict()
    print("\nFirst row of data:",first_row)
    for column_name, values in first_row.items():
        column_values = values[0].split("\x01")
        print(f"{column_name}")
        for column_value in column_values:
            split_values = column_value.split("\x03")
            print(f"split values",split_values)
        #print(f"{column_name}: {values[0]}")
else:
    print("\nThe file is empty.")

# 打印总行数
print(f"\nTotal number of rows: {table.num_rows}")



In [ ]:
import datetime
start_dt = datetime.datetime.now()
print(start_dt)

### 统计每个group_id最大长度

In [ ]:
raw_s3_url = f"s3://{bucket}/mv-poc/raw/"
#raw_s3_url = "s3://mv-mtg-di-for-poc/2024/06/16/08/"
calculate_s3_url = f"s3://{bucket}/mobivista/temp/"

In [ ]:
from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput
)

processing_inputs = [
        ProcessingInput(
            source=raw_s3_url, 
            destination="/opt/ml/processing/input",
            s3_input_mode="File",
            s3_data_distribution_type="ShardedByS3Key"
        )
    ]

processing_outputs = [
        ProcessingOutput(
            output_name="feature_data", 
            source="/opt/ml/processing/output/features",
            destination=calculate_s3_url,
        )
    ]



In [ ]:
configuration = [
    {
        "Classification": "spark-defaults",
        "Properties": {
             "spark.jars":"s3://salunchbucket/jars/spark-tfrecord_2.12-0.3.3.jar",
             "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
             "spark.dynamicAllocation.executorIdleTimeout": 3600,
             "spark.executor.memory": "60G",
             "spark.executor.cores": 40,
             "spark.driver.memory": "80G",
             "spark.driver.cores": 32,
             "spark.dynamicAllocation.initialExecutors":2,
             "spark.sql.hive.convertMetastoreParquet": "true",
             "spark.driver.extraJavaOptions": "-Dlog4j.rootCategory=ERROR,console",
             "spark.executor.extraJavaOptions": "-Dlog4j.rootCategory=ERROR,console"
        },
    }
]



spark_processor = PySparkProcessor(
    base_job_name="spark-feature-lenth",
    framework_version="3.1",
    role=role,
    instance_count=1,
    instance_type="ml.m5.12xlarge",
    #max_runtime_in_seconds=1200,
)


spark_processor.run(
    inputs=processing_inputs,
    outputs=processing_outputs,
    submit_app="./calculate_seq_lenth_v1.py",
    configuration=configuration,
    spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, "pyspark"),
    logs=False,
)


### 统计group_id唯一值的dict

In [ ]:
raw_s3_url = f"s3://{bucket}/mv-poc/raw/"
#raw_s3_url = "s3://mv-mtg-di-for-poc/2024/06/16/08/"
calculate_s3_url = f"s3://{bucket}/mobivista/temp/"

In [ ]:
from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput
)

processing_inputs = [
        ProcessingInput(
            source=raw_s3_url, 
            destination="/opt/ml/processing/input",
            s3_input_mode="File",
            s3_data_distribution_type="ShardedByS3Key"
        )
    ]

processing_outputs = [
        ProcessingOutput(
            output_name="feature_data", 
            source="/opt/ml/processing/output/features",
            destination=calculate_s3_url,
        )
    ]



In [ ]:
configuration = [
    {
        "Classification": "spark-defaults",
        "Properties": {
             "spark.jars":"s3://salunchbucket/jars/spark-tfrecord_2.12-0.3.3.jar",
             "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
             "spark.dynamicAllocation.executorIdleTimeout": 3600,
             "spark.executor.memory": "60G",
             "spark.executor.cores": 40,
             "spark.driver.memory": "80G",
             "spark.driver.cores": 32,
             "spark.dynamicAllocation.initialExecutors":4,
             "spark.sql.hive.convertMetastoreParquet": "true",
             "spark.driver.extraJavaOptions": "-Dlog4j.rootCategory=ERROR,console",
             "spark.executor.extraJavaOptions": "-Dlog4j.rootCategory=ERROR,console"
        },
    }
]



spark_processor = PySparkProcessor(
    base_job_name="spark-feature-dim",
    framework_version="3.1",
    role=role,
    instance_count=2,
    instance_type="ml.m5.12xlarge",
    #max_runtime_in_seconds=1200,
)


spark_processor.run(
    inputs=processing_inputs,
    outputs=processing_outputs,
    submit_app="./calculate_dim_v3.py",
    configuration=configuration,
    spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, "pyspark"),
    logs=False,
)


In [ ]:
!aws s3 cp s3://sagemaker-us-west-2-687912291502/mv-poc/output/part-00000-757be5db-4d45-493f-a7b9-836104157742-c000.json ./ 

In [ ]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/mv-poc/output/ --human-readable

In [ ]:
end_dt = datetime.datetime.now()
print(end_dt)

### 特征编码处理及tfrecord转换

#### extend prebuild pyspark process image

In [ ]:
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

In [2]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-feature-process-mv-poc"

In [ ]:
%%writefile Dockerfile.inference
## You should change below region code to the region you used, here sample is use us-west-2
From 153931337802.dkr.ecr.us-west-2.amazonaws.com/sagemaker-spark-processing:3.0-cpu-py39-v1.0 

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

RUN pip3 install tensorflow==2.13.0


In [ ]:
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 153931337802.dkr.ecr.us-west-2.amazonaws.com

In [ ]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} -f Dockerfile.inference .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}



In [3]:
inference_image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format("687912291502", "us-west-2", repo_name)
inference_image_uri

'687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-feature-process-mv-poc:latest'

#### 小文件合并

In [ ]:
configuration = [
    {
        "Classification": "spark-defaults",
        "Properties": {
             "spark.jars":"s3://salunchbucket/jars/spark-tfrecord_2.12-0.3.3.jar",
             "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
             "spark.dynamicAllocation.executorIdleTimeout": 3600,
             "spark.executor.memory": "40G",
             "spark.executor.cores": 10,
             "spark.driver.memory": "30G",
             "spark.driver.cores": 40,
             "spark.sql.hive.convertMetastoreParquet": "true",
             "spark.driver.extraJavaOptions": "-XX:-PrintGCDetails -XX:-PrintGCTimeStamps -XX:-PrintGCDateStamps -Dlog4j.rootCategory=ERROR,console",
             "spark.executor.extraJavaOptions": "-XX:-PrintGCDetails -XX:-PrintGCTimeStamps -XX:-PrintGCDateStamps -Dlog4j.rootCategory=ERROR,console",
             "spark.hadoop.fs.s3a.connection.maximum": 1000,
             "spark.hadoop.fs.s3a.attempts.maximum":10,
             "spark.hadoop.fs.s3a.retry.limit":10,
             "spark.hadoop.fs.s3a.retry.interval":"1000ms"
             
             
        },
    },
    {
        "Classification": "yarn-site",
        "Properties": {
            "yarn.nodemanager.local-dirs":"/opt/ml/output/tmp/local-dirs/",
            "yarn.nodemanager.log-dirs":"/opt/ml/output/tmp/log-dirs/",
            "yarn.nodemanager.localizer.cache.cleanup.interval-ms": "60000",
            "yarn.nodemanager.localizer.cache.target-size-mb": "15120"
        }
        
    }
]


spark_processor = PySparkProcessor(
    base_job_name="spark-merge-process",
    framework_version="3.0",
    role=role,
    instance_count=6,
    instance_type="ml.m5.24xlarge",
    image_uri=inference_image_uri,
    volume_size_in_gb=1000
)



spark_processor.run(
    submit_app="./feature_merge.py",
    configuration=configuration,
    spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, "pyspark"),
    logs=False,
    arguments=['--num-partitions', '60000']
)

#### 特征转换

In [4]:
raw_s3_url = f"s3://{bucket}/mv-poc/raw3/"
#raw_s3_url = "s3://mv-mtg-di-for-poc/2024/06/16"
feature_hash_dict_url = "s3://sagemaker-us-west-2-687912291502/mv-poc/output/"
features_s3_url = f"s3://{bucket}/mobivista/tf_train/features2"

In [5]:
from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput
)

processing_inputs = [
        ProcessingInput(
            source=feature_hash_dict_url, 
            destination="/opt/ml/processing/input/data/",
            s3_input_mode="File",
            s3_data_distribution_type="ShardedByS3Key"
        ),
        ProcessingInput(
            source=raw_s3_url, 
            destination="/opt/ml/processing/input/data/raw/",
            s3_input_mode="Pipe",
            s3_data_distribution_type="ShardedByS3Key"
        )
     
    ]

processing_outputs = [
        ProcessingOutput(
            output_name="feature_data", 
            source="/opt/ml/processing/output/features/",
            destination=features_s3_url,
        )
    ]



In [ ]:
configuration = [
    {
        "Classification": "spark-defaults",
        "Properties": {
             "spark.jars":"s3://salunchbucket/jars/spark-tfrecord_2.12-0.3.3.jar",
             "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
             "spark.dynamicAllocation.executorIdleTimeout": 3600,
             #"spark.executor.memory": "23G",
             #"spark.executor.cores": 7,
             #"spark.driver.memory": "20G",
             #"spark.driver.cores": 7,
             "spark.executor.cores": 3,
             "spark.executor.memory": "6G",
             "spark.driver.memory": "10G",
             "spark.driver.cores": 6,
             "spark.dynamicAllocation.enabled":"true",
             "spark.shuffle.service.enabled":"true",
             #"spark.dynamicAllocation.initialExecutors":16,
             "spark.sql.hive.convertMetastoreParquet": "true",
             "spark.driver.extraJavaOptions": "-XX:-PrintGCDetails -XX:-PrintGCTimeStamps -XX:-PrintGCDateStamps -Dlog4j.rootCategory=ERROR,console",
             "spark.executor.extraJavaOptions": "-XX:-PrintGCDetails -XX:-PrintGCTimeStamps -XX:-PrintGCDateStamps -Dlog4j.rootCategory=ERROR,console",
             "spark.sparkContext.setLogLevel": "ERROR",
             "spark.sql.streaming.metricsEnabled": "false",            
             "spark.hadoop.fs.s3a.connection.maximum": 1000,
             "spark.hadoop.fs.s3a.attempts.maximum":10,
             "spark.hadoop.fs.s3a.retry.limit":10,
             "spark.hadoop.fs.s3a.retry.interval":"1000ms"
             
             
        },
    },
    {
        "Classification": "yarn-site",
        "Properties": {
            "yarn.nodemanager.local-dirs":"/opt/ml/output/tmp/local-dirs/",
            "yarn.nodemanager.log-dirs":"/opt/ml/output/tmp/log-dirs/",
            "yarn.nodemanager.localizer.cache.cleanup.interval-ms": "20000",
            "yarn.nodemanager.localizer.cache.target-size-mb": "152120"
        }
        
    }

]


spark_processor = PySparkProcessor(
    base_job_name="spark-feature-process",
    framework_version="3.0",
    role=role,
    instance_count=8,
    instance_type="ml.m5.24xlarge",
    image_uri=inference_image_uri,
    volume_size_in_gb=4000
)



spark_processor.run(
    #inputs=processing_inputs,
    outputs=processing_outputs,
    submit_app="./feature_spark_v8.py",
    configuration=configuration,
    spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, "pyspark"),
    logs=False,
    arguments=['--num-partitions', '1000',
              '--padding-lenth','10'],
    #submit_py_files=["s3://sagemaker-us-west-2-687912291502/env/tensorflow_env.zip"]
)


INFO:sagemaker:Creating processing-job with name spark-feature-process-2024-07-23-23-51-35-561


................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................_______________________________________________s